In [ ]:
!pip install stable-baselines3

In [ ]:
!pip install Shimmy

1. Setup Mario

In [ ]:
# import the game
import gym_super_mario_bros

# import the joypad wrapper
from nes_py.wrappers import JoypadSpace

#import simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

#import gym module
import gym

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
# setup game
env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human")
env=JoypadSpace(env,SIMPLE_MOVEMENT)

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
env.action_space.sample()
SIMPLE_MOVEMENT[env.action_space.sample()]

In [ ]:
env.observation_space.shape

In [ ]:
# create a flag - restart or not
done=True
# loop through each frame
for step in range(1000):
    if done:
#       start the game
        env.reset()
#     take a random action
    state,reward,done,truncate,info=env.step(env.action_space.sample())
#     print(truncate)
#     show the game on the screen
#     env.render()
#     close the game
env.close()

In [ ]:
a=env.reset()
b=env.step(1)
print(b)

In [ ]:
env.close()

2. Preprocess Environment

In [ ]:
#import frame stacker wrapper
from gym.wrappers import GrayScaleObservation


#import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# import plot
from matplotlib import pyplot as plt

In [ ]:
# create base environment
env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True)

#1. simplify controls via joypadspace wrapper
env=JoypadSpace(env,SIMPLE_MOVEMENT)
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)

#2. grayscale image frame via grayscale wrapper
env=GrayScaleObservation(env,keep_dim=True)

#3. Wrap inside dummy environment, so the data can be processed in a stablebaselines 3 env
env=DummyVecEnv([lambda: env])

#4. Wrap inside framestacker
env=VecFrameStack(env,4,channels_order='last')

In [ ]:
state=env.reset()

In [ ]:
state.shape

In [ ]:
# run at least 4 times
state,reward,done,info=env.step([env.action_space.sample()])

In [ ]:
plt.imshow(state[0])

In [ ]:
plt.figure(figsize=(10,8))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

In [ ]:
env.close()    

3.Train the RL model

In [ ]:
#import os for file path management
import os
#import PPO for algos
from stable_baselines3 import PPO
import torch
torch.cuda.empty_cache()

#import Base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
# pretty self explanatory from the name
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
#setup model callback
callback=TrainAndLoggingCallback(check_freq=1000000,save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model being built
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=256) 

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=250000, callback=callback)